## TMM panel simulator adapted for honeycomb cavities and exterior necks

### Author: Alex Fanomezantsoa Rabearivony
### 2025

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; color: white; text-align: center;
            margin: 20px 0;">
    <h1>Acoustic Metamaterial Simulator</h1>
    <h3>Recycled Plastic Bottles - Helmholtz Resonators</h3>
    <g5>Interactive research tool for acoustic performance analysis</g5>
</div>

<div style="margin-left: 50px;">
<img src="IMAGES/3D honey.png" width="65%">
</div>

## Helmholtz Resonator Array – Interactive Simulation (TMM)

In this notebook, we investigate the **acoustic performance of a network of Helmholtz resonators** (plastic bottles) using the **Transfer Matrix Method (TMM)**.  
This approach allows us to simulate the **absorption coefficient** ($\alpha$) and the **transmission loss** (TL, in dB) of a periodic array of bottles.

---

### Theoretical Background

A **single Helmholtz resonator** has a resonance frequency given by:

$$
f_0 = \frac{c}{2 \pi} \sqrt{\frac{A}{V \, L_\text{eff}}}
$$

where:  
- $c$ = speed of sound in air (≈ 343 m/s)  
- $A = \pi r^2$ = neck cross-sectional area [m²]  
- $V$ = cavity volume [m³]  
- $L_\text{eff} = L + 1.7r$ = effective neck length [m]  

This expression provides the **theoretical resonance frequency** without losses.

---

### Absorption and Transmission Loss

For an incident plane wave, the **reflection coefficient** $R$ and the **transmission coefficient** $T$ can be obtained from the resonator impedance $Z$:

$$
R = \frac{Z - Z_0}{Z + Z_0}, \quad 
T = \frac{2Z_0}{Z + Z_0}
$$

with $Z_0 = \rho_0 c_0$ the characteristic impedance of air.  
Then:

- **Absorption coefficient**:  
$$
\alpha = 1 - |R|^2 - |T|^2
$$  

- **Transmission loss (TL)** in decibels:  
$$
TL = -20 \log_{10} |T|
$$  

---

### Analytical vs. Observed Resonance

 The **theoretical resonance frequency** $f_0$ does not always coincide with the **observed peak** of absorption or transmission loss.  
This is because **loss mechanisms** (viscous, thermal, radiation) and **array coupling effects** shift the resonance.

- The **red dashed line** on the plots indicates the **analytical $f_0$**.  
- The **measured peak** of $\alpha$ or TL may occur at a slightly different frequency, showing the impact of **damping and array interactions**.

---

## Limits of the **Transfer-Matrix Method (TMM) used here**

The TMM implementation in this notebook models each column as a cascade of propagation sections and a shunt admittance representing the resonators. This is a pragmatic and computationally efficient approach but carries specific limitations:

1. **One-dimensional (plane-wave) assumption in each propagation segment.** The TMM assumes plane waves propagate between scattering/shunt sites; evanescent higher transverse modes are neglected.  
2. **Heuristic coupling / homogenization.** The treatment of a column admittance as `rows * Y_hr * FF` is a pragmatic scaling (filling factor). It approximates coupling but does not fully account for near-field evanescent coupling or inter-resonator scattering.  
3. **Finite-size and edge effects.** TMM typically represents an infinite or periodically repeated geometry; finite panel edges can introduce diffraction and edge scattering that TMM does not capture accurately.  
4. **Single-frequency linear analysis.** The method computes linear steady-state responses; strongly nonlinear behavior (flow separation, vortex shedding at the neck) is outside scope.  
5. **Radiation and mounting conditions.** Radiation impedance and panel mounting (flexible backing, poroelastic supports) are simplified or absent; these can noticeably change absorption/TL in practice.  
6. **Numerical conditioning and singularities.** Near ideal (lossless) resonances the matrices can become ill-conditioned unless realistic damping (R0, viscous terms) or numerical regularization is included.  
7. **Parameter calibration required for quantitative predictions.** R0, Rscale and the filling factor FF are phenomenological in this model — for publication-grade predictions calibrate them by measurement or high-fidelity simulation.

---

### Key Observations

1. **Single bottle** → very low absorption (narrowband, small $\alpha$ values).  
2. **Array of bottles** → stronger collective effect, but resonance remains **narrowband**.  
3. **Losses** (via $R_0$ or viscous scaling) are necessary to reach significant absorption levels.  
4. **TL curve** is complementary to absorption: when $\alpha$ peaks, TL also increases.  

---
## Legend — variables controlled by widgets

<div align="center">
<img src="IMAGES/top honey.png" width="60%">
</div>
-
<div align="center">
<img src="IMAGES/Cross-section honey.png" width="38%">
</div>

- **cols** : number of columns in the array (integer).  
- **rows** : number of rows in the array (integer).  
- **neck r (cm)** : neck radius of each resonator, displayed in centimetres (converted to m for calculation).  
- **neck L (cm)** : neck (geometric) length, displayed in centimetres (converted to m for calculation).  
- **cav-side (mm)** : length of one side of the square cavity, defined in millimetres (converted to metres for computation).
- **area ratio** : ratio between the octagonal and square cavity areas; controls geometric scaling to keep seamless tiling without gaps.
- **fmax (Hz)** : maximum frequency shown in the plot (display zoom only).  
- **R0 (Pa·s/m)** : frequency-independent lumped resistance used to represent losses (user tuneable).  
- **R scale** : multiplicative scaling of the viscous resistance model.  
- **M_mem (mg)** : optional added mass (membrane mass) in milligrams.

This interactive setup allows exploring **how geometry and material properties influence absorption and TL**.  

---

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import csv
import os

# --- physical constants ---
rho0 = 1.21
c0 = 343.0
Z0 = rho0 * c0
mu = 1.84e-5

# --- helper functions ---
def helmholtz_f0(A, V, L, r, c=c0):
    L_eff = L + 1.7*r
    return (c/(2*np.pi)) * np.sqrt( A / (V * L_eff) )

def viscous_delta(f):
    omega = 2*np.pi*np.maximum(f,1e-8)
    return np.sqrt(2*mu/(rho0*omega))

def R_viscous(f, r):
    delta = viscous_delta(f)
    r = np.maximum(r, 1e-6)
    return Z0 * 2.0 * (delta / r)

def Z_helmholtz(f, A, V, L, r, R0=0.0, Rscale=1.0, mem_mass_kg=0.0):
    """Impedance of one HR (vectorized)."""
    omega = 2*np.pi*np.maximum(f,1e-8)
    L_eff = L + 1.7*r
    M_air = rho0 * L_eff / A
    M_extra = 0.0
    if mem_mass_kg > 0:
        M_extra = mem_mass_kg / (A + 1e-12)
    M = M_air + M_extra
    C = V / (rho0 * c0**2)
    Rv = Rscale * R_viscous(f, r)
    Z = R0 + Rv + 1j*omega*M - 1j/(omega*C)
    return Z

def compute_array_response(freq, rows, cols, d, A_neck, V_sq, V_oct, L_neck, r_neck,
                           R0, Rscale, mem_mass_kg):
    omega = 2*np.pi*freq
    k0 = omega / c0
    
    # Impedance for square and octogone cavities
    Z_sq = Z_helmholtz(freq, A_neck, V_sq, L_neck, r_neck, R0=R0, Rscale=Rscale, mem_mass_kg=mem_mass_kg)
    Z_oct = Z_helmholtz(freq, A_neck, V_oct, L_neck, r_neck, R0=R0, Rscale=Rscale, mem_mass_kg=mem_mass_kg)
    
    # Combine impedances in parallel (per row)
    Y_row = 1.0 / (Z_sq + 1e-18) + 1.0 / (Z_oct + 1e-18)
    Y_col = rows * Y_row
    kd = k0 * (d/2.0)
    cosk = np.cos(kd); sink = np.sin(kd)
    P11 = cosk; P12 = 1j * Z0 * sink
    P21 = 1j * (1.0/Z0) * sink; P22 = cosk
    
    alpha = np.zeros_like(freq, dtype=float)
    TL = np.zeros_like(freq, dtype=float)
    
    for i in range(len(freq)):
        P = np.array([[P11[i], P12[i]],[P21[i], P22[i]]], dtype=complex)
        S = np.array([[1.0, 0.0],[Y_col[i], 1.0]], dtype=complex)
        Mcell = P @ S @ P
        Mglob = np.linalg.matrix_power(Mcell, cols)
        A = Mglob[0,0]; B = Mglob[0,1]; Cmat = Mglob[1,0]; D = Mglob[1,1]
        T = 2.0 / (A + B/Z0 + Cmat*Z0 + D)
        R = (A + B/Z0 - Cmat*Z0 - D) / (A + B/Z0 + Cmat*Z0 + D)
        TL[i] = -20.0*np.log10(np.abs(T) + 1e-20)
        TL[i] = np.clip(TL[i], 0, 120)  # plafonner à 120 dB
        alpha[i] = np.clip(1.0 - np.abs(R)**2 - np.abs(T)**2, 0.0, 1.0)
        
    return alpha, TL, Z_sq, Z_oct

# ----------------- widgets -----------------
cols_w = widgets.IntSlider(value=15, min=1, max=24, step=1, description='cols', continuous_update=False)
rows_w = widgets.IntSlider(value=15, min=1, max=24, step=1, description='rows', continuous_update=False)
neck_r_w = widgets.FloatSlider(value=10.0, min=1.0, max=20.0, step=0.5, description='neck r (mm)', continuous_update=False)
neck_L_w = widgets.FloatSlider(value=10.0, min=5.0, max=50.0, step=1.0, description='neck L (mm)', continuous_update=False)
side_square_w = widgets.FloatSlider(value=66.0, min=20.0, max=100.0, step=1.0, description='side-sqr(mm)', continuous_update=False)
area_ratio_w = widgets.FloatSlider(value=1.08, min=1.02, max=1.15, step=0.01, description='area ratio', continuous_update=False)
R0_w = widgets.FloatSlider(value=415.0, min=0.0, max=500.0, step=5.0, description='R0', continuous_update=False)
Rscale_w = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='R scale', continuous_update=False)
mass_w = widgets.FloatSlider(value=0.0, min=0.0, max=500.0, step=1.0, description='M_mem (mg)', continuous_update=False)
fmax_w = widgets.IntSlider(value=1000, min=200, max=8000, step=100, description='fmax (Hz)', continuous_update=False)

run_btn = widgets.Button(description='Run simulation', button_style='success')
reset_btn = widgets.Button(description='Reset', button_style='warning')

out = widgets.Output(layout={'border':'1px solid black'})

# internal fixed calc grid
NPOINTS = 3000

# --- Export CSV function ---
def export_csv(params):
    filename = "Honeycomb_results.csv"
    file_exists = os.path.isfile(filename)
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['side_square(mm)','neck r(mm)','neck L(mm)','R0','R scale','M_mem(mg)','alpha_max','TL_max','f0(Hz)'])
        writer.writerow(params)

# --- on_run function ---
def on_run(b):
    with out:
        clear_output(wait=True)
        # read widgets
        cols = cols_w.value
        rows = rows_w.value
        r_neck = neck_r_w.value / 1000.0  # mm → m
        L_neck = neck_L_w.value / 1000.0  # mm → m
        side_square = side_square_w.value / 1000.0  # mm → m
        area_ratio = area_ratio_w.value
        R0 = R0_w.value
        Rscale = Rscale_w.value
        mem_mass_kg = mass_w.value * 1e-6
        d = side_square  # distance inter HR (approx)

        # --- Calculate octagon chamfer x from area_ratio ---
        # Solve 2*x^2 + 4*a*x + a^2*(1 - k) = 0
        a = side_square
        k = area_ratio
        discrim = 4*a**2 - 8*a**2*(1 - k)/2
        x = (-2*a + np.sqrt(4*a**2 - 8*a**2*(1 - k)/2))/4  # simplified positive root
        x = max(x, 0)  # ensure non-negative

        # Volumes
        V_sq = a**2 * d  # m^3
        V_oct = (a + 2*x)**2 - 2*x**2
        V_oct *= d  # m^3
        A_neck = np.pi * r_neck**2

        # Estimate resonance frequency for info
        f_res_est = helmholtz_f0(A_neck, V_sq, L_neck, r_neck)

        fcalc_upper = max(int(np.ceil(4*f_res_est)), 8000)
        freq = np.linspace(0.1, fcalc_upper, NPOINTS)

        alpha, TL, Z_sq, Z_oct = compute_array_response(freq, rows, cols, d, A_neck, V_sq, V_oct, L_neck, r_neck, R0, Rscale, mem_mass_kg)

        # display range
        fmax = fmax_w.value
        idx = freq <= fmax
        f_show = freq[idx]; alpha_show = alpha[idx]; TL_show = TL[idx]

        alpha_max = np.max(alpha); f_alpha = freq[np.argmax(alpha)]
        TL_max = np.max(TL); f_TL = freq[np.argmax(TL)]
        import csv
        import os

        # --- CSV export ---
        csv_file = "Honeycomb_results.csv"
        file_exists = os.path.isfile(csv_file)
        
        with open(csv_file, mode='a', newline='') as f:
            writer = csv.writer(f)
            if not file_exists:
                # écrire l'en-tête si le fichier n'existe pas
                writer.writerow(["side-sqr(mm)", "area_ratio", "neck r (mm)", "neck L (mm)", 
                                 "R0", "R scale", "M_mem (mg)", "alpha_max", "TL_max", "fo(Hz)"])
            # écrire la ligne du run actuel
            writer.writerow([side_square_w.value,
                             area_ratio_w.value,
                             neck_r_w.value,
                             neck_L_w.value,
                             R0_w.value,
                             Rscale_w.value,
                             mass_w.value,
                             np.round(alpha_max,3),
                             np.round(TL_max,3),
                             np.round(f_res_est,1)])
        
        # --- Plot ---
        fig, ax = plt.subplots(1,2, figsize=(12,4))
        ax[0].plot(f_show, alpha_show, lw=1.4)
        ax[0].axvline(f_res_est, color='r', ls='--', label=f"single f0={f_res_est:.1f} Hz")
        ax[0].set_xlabel('Freq (Hz)'); ax[0].set_ylabel('Absorption α'); ax[0].set_ylim(-0.02,1.05); ax[0].grid(True)
        ax[0].legend(); ax[0].set_title(f"α_max={alpha_max:.2f} at {f_alpha:.0f} Hz")

        ax[1].plot(f_show, TL_show, lw=1.4)
        ax[1].axvline(f_res_est, color='r', ls='--')
        ax[1].set_xlabel('Freq (Hz)'); ax[1].set_ylabel('Transmission Loss (dB)'); ax[1].grid(True)
        ax[1].set_title(f"TL_max={TL_max:.1f} dB at {f_TL:.0f} Hz")
        plt.suptitle(f"Array: cols={cols}, rows={rows}, neck r={neck_r_w.value:.1f} mm, L={neck_L_w.value:.1f} mm, side_square={side_square_w.value:.1f} mm")
        plt.show()

        # --- Export CSV ---
        export_csv([side_square_w.value, neck_r_w.value, neck_L_w.value, R0, Rscale, mass_w.value, alpha_max, TL_max, f_res_est])

        print(f"Single HR approx f0 = {f_res_est:.1f} Hz")
        print(f"Loss model: R0={R0:.1f} Pa·s/m, Rscale={Rscale:.2f}, mem mass={mass_w.value:.1f} mg")
        print(f"Alpha_max={alpha_max:.2f}, TL_max={TL_max:.1f} dB (plafonné à 120 dB)")

# --- Reset handler ---
def on_reset(b):
    cols_w.value = 15
    rows_w.value = 15
    neck_r_w.value = 10.0
    neck_L_w.value = 10.0
    side_square_w.value = 66.0
    area_ratio_w.value = 1.08
    R0_w.value = 415.0
    Rscale_w.value = 1.0
    mass_w.value = 0.0
    fmax_w.value = 1000
    with out:
        clear_output(wait=True)
        print("🔄 Parameters reset to default values.")

# --- Layout ---
controls = widgets.VBox([
    widgets.HBox([cols_w, rows_w, mass_w]),
    widgets.HBox([neck_r_w, neck_L_w, fmax_w]),
    widgets.HBox([side_square_w, area_ratio_w, run_btn]),
    widgets.HBox([R0_w, Rscale_w, reset_btn]),
])

display(controls, out)

run_btn.on_click(on_run)
reset_btn.on_click(on_reset)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…